<a target="_blank" href="https://colab.research.google.com/github/tomas-gajarsky/facetorch/blob/main/notebooks/facetorch_notebook_demo.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# Facetorch notebook demo


## Check GPU availability

In [9]:
# Check GPU availability
!nvidia-smi
# Edit -> Notebook settings -> Hardware accelerator



Sat Jan 28 13:57:14 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0    26W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Add cell timer to the notebook

In [10]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 3.14 s (started: 2023-01-28 13:57:14 +00:00)


## Install dependencies

In [11]:
!python --version
!python -m pip install --upgrade pip
!python -m pip install facetorch>=0.2.0

!pip list | grep facetorch

Python 3.8.10
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
facetorch                     0.2.0
time: 10 s (started: 2023-01-28 13:57:17 +00:00)


## Download config and image

In [12]:
!wget -O ./test.jpg https://github.com/tomas-gajarsky/facetorch/blob/main/data/input/test.jpg?raw=true
!wget -O ./gpu.config.yml https://raw.githubusercontent.com/tomas-gajarsky/facetorch/main/conf/merged/gpu.merged.config.yaml

--2023-01-28 13:57:27--  https://github.com/tomas-gajarsky/facetorch/blob/main/data/input/test.jpg?raw=true
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/tomas-gajarsky/facetorch/raw/main/data/input/test.jpg [following]
--2023-01-28 13:57:28--  https://github.com/tomas-gajarsky/facetorch/raw/main/data/input/test.jpg
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tomas-gajarsky/facetorch/main/data/input/test.jpg [following]
--2023-01-28 13:57:28--  https://raw.githubusercontent.com/tomas-gajarsky/facetorch/main/data/input/test.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443...

## Import packages

In [13]:
from facetorch import FaceAnalyzer
from omegaconf import OmegaConf
from torch.nn.functional import cosine_similarity
from typing import Dict
import operator


time: 1.46 ms (started: 2023-01-28 13:57:28 +00:00)


## Configure

In [14]:
path_image="./test.jpg"
path_config="gpu.config.yml"


cfg = OmegaConf.load(path_config)

time: 83.6 ms (started: 2023-01-28 13:57:28 +00:00)


## Startup

In [16]:
# initialize
analyzer = FaceAnalyzer(cfg.analyzer)

# warmup
response = analyzer.run(
        path_image=path_image,
        batch_size=cfg.batch_size,
        fix_img_size=cfg.fix_img_size,
        return_img_data=False,
        include_tensors=True,
        path_output=None,
    )

{"asctime": "2023-01-28 13:58:21,980", "levelname": "INFO", "message": "Initializing FaceAnalyzer"}
{"asctime": "2023-01-28 13:58:21,982", "levelname": "INFO", "message": "Initializing BaseReader"}
{"asctime": "2023-01-28 13:58:22,003", "levelname": "INFO", "message": "Initializing FaceDetector"}
{"asctime": "2023-01-28 13:58:22,232", "levelname": "INFO", "message": "Initializing FaceUnifier"}
{"asctime": "2023-01-28 13:58:22,250", "levelname": "INFO", "message": "Initializing FacePredictor objects"}
{"asctime": "2023-01-28 13:58:22,252", "levelname": "INFO", "message": "Initializing FacePredictor embed"}
{"asctime": "2023-01-28 13:58:22,426", "levelname": "INFO", "message": "Initializing FacePredictor verify"}
{"asctime": "2023-01-28 13:58:23,713", "levelname": "INFO", "message": "Initializing FacePredictor fer"}
{"asctime": "2023-01-28 13:58:23,934", "levelname": "INFO", "message": "Initializing FacePredictor deepfake"}
{"asctime": "2023-01-28 13:58:24,521", "levelname": "INFO", "mes

time: 15.5 s (started: 2023-01-28 13:58:21 +00:00)


## Inference

In [17]:
response = analyzer.run(
        path_image=path_image,
        batch_size=cfg.batch_size,
        fix_img_size=cfg.fix_img_size,
        return_img_data=False,
        include_tensors=True,
        path_output=None,
    )

{"asctime": "2023-01-28 13:58:37,530", "levelname": "INFO", "message": "Running FaceAnalyzer"}
{"asctime": "2023-01-28 13:58:37,536", "levelname": "INFO", "message": "Reading image", "path_image": "./test.jpg"}
{"asctime": "2023-01-28 13:58:37,552", "levelname": "INFO", "message": "Detecting faces"}
{"asctime": "2023-01-28 13:58:44,952", "levelname": "INFO", "message": "Number of faces: 4"}
{"asctime": "2023-01-28 13:58:44,953", "levelname": "INFO", "message": "Unifying faces"}
{"asctime": "2023-01-28 13:58:44,959", "levelname": "INFO", "message": "Predicting facial features"}
{"asctime": "2023-01-28 13:58:44,960", "levelname": "INFO", "message": "Running FacePredictor: embed"}
{"asctime": "2023-01-28 13:58:51,305", "levelname": "INFO", "message": "Running FacePredictor: verify"}
{"asctime": "2023-01-28 13:58:58,707", "levelname": "INFO", "message": "Running FacePredictor: fer"}
{"asctime": "2023-01-28 13:59:12,363", "levelname": "INFO", "message": "Running FacePredictor: deepfake"}
{"

time: 49.1 s (started: 2023-01-28 13:58:37 +00:00)


## Facial Expressions

In [18]:
{face.indx: face.preds["fer"].label for face in response.faces}


{0: 'Happiness', 1: 'Surprise', 2: 'Happiness', 3: 'Disgust'}

time: 7.51 ms (started: 2023-01-28 13:59:26 +00:00)


## Embedding cosine similarity

In [19]:
def compute_embed_similarity(predictor_name: str = "verify", base_face_id: int = 0) -> Dict:
  base_emb = response.faces[base_face_id].preds[predictor_name].logits
  sim_dict = {face.indx: cosine_similarity(base_emb, face.preds[predictor_name].logits, dim=0).item() for face in response.faces}
  sim_dict_sorted = dict(sorted(sim_dict.items(), key=operator.itemgetter(1),reverse=True))
  return sim_dict_sorted

time: 1.01 ms (started: 2023-01-28 13:59:26 +00:00)


### Face representation learning

In [20]:
compute_embed_similarity(predictor_name="embed")

{0: 1.0,
 3: 0.012771518900990486,
 2: -0.012598775327205658,
 1: -0.012680798768997192}

time: 10.1 ms (started: 2023-01-28 13:59:26 +00:00)


### Face verification

In [21]:
compute_embed_similarity(predictor_name="verify")

{0: 1.0000001192092896,
 1: 0.11373738944530487,
 3: 0.07457146048545837,
 2: -0.06798631697893143}

time: 4.85 ms (started: 2023-01-28 13:59:26 +00:00)


## Full response

In [22]:
response

Response(faces=[Face(indx=0, loc=Location(x1=523, x2=644, y1=330, y2=452), dims=Dimensions(height=122, width=121), tensor=tensor([[[0.8784, 0.8784, 0.8784,  ..., 0.6774, 0.6588, 0.6588],
         [0.8784, 0.8784, 0.8784,  ..., 0.6774, 0.6588, 0.6588],
         [0.8784, 0.8784, 0.8784,  ..., 0.6680, 0.6470, 0.6470],
         ...,
         [0.6625, 0.6625, 0.6566,  ..., 0.2169, 0.2196, 0.2196],
         [0.6863, 0.6863, 0.6747,  ..., 0.2173, 0.2196, 0.2196],
         [0.6863, 0.6863, 0.6747,  ..., 0.2173, 0.2196, 0.2196]],

        [[0.6471, 0.6471, 0.6471,  ..., 0.4592, 0.4314, 0.4314],
         [0.6471, 0.6471, 0.6471,  ..., 0.4592, 0.4314, 0.4314],
         [0.6471, 0.6471, 0.6471,  ..., 0.4610, 0.4314, 0.4314],
         ...,
         [0.5841, 0.5841, 0.5781,  ..., 0.0679, 0.0706, 0.0706],
         [0.6078, 0.6078, 0.5962,  ..., 0.0683, 0.0706, 0.0706],
         [0.6078, 0.6078, 0.5962,  ..., 0.0683, 0.0706, 0.0706]],

        [[0.5765, 0.5765, 0.5765,  ..., 0.3993, 0.3725, 0.3725],
 

time: 105 ms (started: 2023-01-28 13:59:26 +00:00)
